In [1]:
!pip install mediapipe

In [2]:
!pip install SpeechRecognition

In [3]:
!pip install pyaudio

In [4]:
!pip install wave

In [5]:
!pip install nltk

In [1]:
import cv2
import mediapipe as mp
import speech_recognition as sr
import pyaudio
import wave
import threading
import os
import nltk

nltk.download('stopwords')

class Face:
    def __init__(self):
        self.bbox = None
        self.img = None
        self.name = None
        self.distance = None
        self.confidence = None
        self.landmarks = None
        self.mouth = None
        self.head = None
        self.eye = None
        self.spoof = None
        self.spoof_score = None

def get_face(frame, bbox):
    real_h, real_w, c = frame.shape
    x, y, w, h = bbox
    y1 = 0 if y < 0 else y
    x1 = 0 if x < 0 else x
    y2 = real_h if y1 + h > real_h else y + h
    x2 = real_w if x1 + w > real_w else x + w
    face = frame[y1:y2, x1:x2, :]
    return face

def detect_faces(frame, confidence=0.7):
    faces = []
    mp_face_detection = mp.solutions.face_detection

    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=confidence) as face_detector:
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        results = face_detector.process(frame)
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    if results.detections:
        for _, detection in enumerate(results.detections):
            face = Face()
            bbox = detection.location_data.relative_bounding_box
            ih, iw, ic = frame.shape
            bbox = (
                int(bbox.xmin * iw),
                int(bbox.ymin * ih),
                int(bbox.width * iw),
                int(bbox.height * ih),
            )
            face.bbox = bbox
            face.confidence = detection.score
            face.img = get_face(frame, face.bbox)
            faces.append(face)

    return faces

def recorder():
    def read_audio(stream, filename):
        chunk = 1024
        sample_format = pyaudio.paInt16
        channels = 2
        fs = 44100
        seconds = 10
        filename = filename
        frames = []

        for i in range(0, int(fs / chunk * seconds)):
            data = stream.read(chunk)
            frames.append(data)

        wf = wave.open(filename, 'wb')
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(sample_format))
        wf.setframerate(fs)
        wf.writeframes(b''.join(frames))
        wf.close()
        stream.stop_stream()
        stream.close()

    def convert(i):
        if i >= 0:
            sound = 'record' + str(i) + '.wav'
            r = sr.Recognizer()

            with sr.AudioFile(sound) as source:
                r.adjust_for_ambient_noise(source)
                print("Converting Audio To Text and saving to file..... ")
                audio = r.listen(source)
            try:
                value = r.recognize_google(audio)
                os.remove(sound)
                if isinstance(value, bytes):
                    result = u"{}".format(value).encode("utf-8")
                else:
                    result = "{}".format(value)

                with open("test.txt", "a") as f:
                    f.write(result)
                    f.write(" ")

            except sr.UnknownValueError:
                print("Speech Recognition could not understand audio")
            except sr.RequestError as e:
                print("Could not request results from Google Speech Recognition service; {0}".format(e))
            except KeyboardInterrupt:
                pass

    p = pyaudio.PyAudio()
    chunk = 1024
    sample_format = pyaudio.paInt16
    channels = 2
    fs = 44100

    def save_audios(i):
        stream = p.open(format=sample_format, channels=channels, rate=fs, frames_per_buffer=chunk, input=True)
        filename = 'record' + str(i) + '.wav'
        read_audio(stream, filename)

    flag = False
    for i in range(100 // 50):
        t1 = threading.Thread(target=save_audios, args=[i])
        x = i - 1
        t2 = threading.Thread(target=convert, args=[x])
        t1.start()
        t2.start()
        t1.join()
        t2.join()
        if i == 2:
            flag = True
    if flag:
        convert(i)
        p.terminate()

if __name__ == "__main__":
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        faces = detect_faces(frame)

        if faces:
            for face in faces:
                bbox = face.bbox
                cv2.rectangle(
                    frame,
                    (bbox[0], bbox[1]),
                    (bbox[0] + bbox[2], bbox[1] + bbox[3]),
                    (0, 255, 0),
                    2,
                )

            if len(faces) > 1:
                print("Suspicious activity detected")
                recorder()

        else:
            print("No faces detected")

        cv2.imshow("Webcam", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sarad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Suspicious activity detected
Converting Audio To Text and saving to file..... 
Speech Recognition could not understand audio
